# Import graspi_igraph

Ensure that you are only using python versions higher than 3.7
- Python version can be checked by running *python --version* in the terminal
- it can also be checked on notebook via *!pip --version* in the code section of the notebook

In [ ]:
import time

In [1]:
import new_igraph_testing as ig
import descriptors as ds
import os
import time

current_dir = os.getcwd()
data_path = f"{current_dir}/graspi_igraph/data/"
descriptors_path = f"{current_dir}/graspi_igraph/p1descriptors/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5
for test_file in test_files:
    # g = ig.generateGraph(data_path + test_file + ".txt")
    import time
    graph_start = time.time()
    g,is_2d= ig.generateGraphAdj(data_path + test_file + ".txt")
    graph_end = time.time()
    stats = ds.desciptors(g)
    #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    with open(descriptors_path + "p1descriptors." + test_file + ".log") as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
    time = stats["time"]
    mem = stats["mem"]
    graph_time = graph_end-graph_start
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {time} second(s)")
    print(f"Peak memory usage for descriptor calculation: {mem} bytes")
    print(stats)
    print("")

data_0.615_4.0_000160 Results
STAT_n passed
STAT_e passed
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_f_D passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_n_D_adj_An passed
CT_n_A_adj_Ca passed
Total time to calculate graph: 9.21011734008789 second(s)
Total time to calculate descriptors: 2.193685531616211 second(s)
Peak memory usage for descriptor calculation: 319244 bytes
{'STAT_n': 40501, 'STAT_e': 3523, 'STAT_n_D': 16223, 'STAT_n_A': 24278, 'STAT_CC_D': 43, 'STAT_CC_A': 1, 'STAT_CC_D_An': 7, 'STAT_CC_A_Ca': 1, 'ABS_f_D': 0.400558, 'CT_f_conn_D_An': 0.134192, 'CT_f_conn_A_Ca': 1.0, 'CT_n_D_adj_An': 147, 'CT_n_A_adj_Ca': 197, 'time': 2.193685531616211, 'mem': 319244}

data_0.5_2.6_000160 Results
STAT_n passed
STAT_e passed
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_f_D passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_n_D_adj_An passed
CT_n_A_ad

In [3]:
import withoutP_igraph_testing as ig
import descriptors as ds
import os

current_dir = os.getcwd()
data_path = f"{current_dir}/graspi_igraph/data/"
descriptors_path = f"{current_dir}/graspi_igraph/descriptors/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5
for test_file in test_files:
    # g = ig.generateGraph(data_path + test_file + ".txt")
    import time
    graph_start = time.time()
    g, is_2d= ig.generateGraphAdj(data_path + test_file + ".txt")
    graph_end = time.time()
    stats = ds.desciptors(g)
    #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    with open(descriptors_path + "descriptors." + test_file + ".log") as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
    time = stats["time"]
    mem = stats["mem"]
    graph_time = graph_end-graph_start
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {time}second(s)")
    print(f"Peak memory usage for descriptor calculation: {mem}bytes")
    print(stats)
    print("")

data_0.615_4.0_000160 Results
STAT_n passed
STAT_e passed
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_f_D passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_n_D_adj_An passed
CT_n_A_adj_Ca passed
Total time to calculate graph: 8.386402606964111 second(s)
Total time to calculate descriptors: 8.251307249069214second(s)
Peak memory usage for descriptor calculation: 51162914bytes
{'STAT_n': 40501, 'STAT_e': 3466, 'STAT_n_D': 16223, 'STAT_n_A': 24278, 'STAT_CC_D': 46, 'STAT_CC_A': 1, 'STAT_CC_D_An': 8, 'STAT_CC_A_Ca': 1, 'ABS_f_D': 0.400558, 'CT_f_conn_D_An': 0.134192, 'CT_f_conn_A_Ca': 1.0, 'CT_n_D_adj_An': 147, 'CT_n_A_adj_Ca': 197, 'time': 8.251307249069214, 'mem': 51162914}

data_0.5_2.6_000160 Results
STAT_n passed
STAT_e passed
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_f_D passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_n_D_adj_An passed
CT_n_A

In [2]:
import combined_with as ig
import new_descriptors as ds
import os
import time

current_dir = os.getcwd()
data_path = f"{current_dir}/graspi_igraph/data/"
descriptors_path = f"{current_dir}/graspi_igraph/p1descriptors/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5
for test_file in test_files:
    if 'data_4_3_2' in test_file or 'data_5x4x3' in test_file:
        continue
    # g = ig.generateGraph(data_path + test_file + ".txt")
    import time
    graph_start = time.time()
    g,is_2D, black_vertices, white_vertices, black_green, black_interface_red, white_interface_blue, dim, interface_edge_comp_paths, shortest_path_to_red, shortest_path_to_blue, CT_n_D_adj_An, CT_n_A_adj_Ca= ig.generateGraph(data_path + test_file + ".txt")
    graph_end = time.time()
    stats = ds.descriptors(g, test_file, black_vertices, white_vertices, black_green, black_interface_red, white_interface_blue, dim, interface_edge_comp_paths, shortest_path_to_red, shortest_path_to_blue, CT_n_D_adj_An, CT_n_A_adj_Ca)
    #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    with open(descriptors_path + "p1descriptors." + test_file + ".log") as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
    time = stats["time"]
    mem = stats["mem"]
    graph_time = graph_end-graph_start
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {time} second(s)")
    print(f"Peak memory usage for descriptor calculation: {mem} bytes")
    print(stats)
    print("")

data_0.615_4.0_000160 Results
STAT_n passed
STAT_e passed
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_f_D passed
DISS_wf10_D failed - 0.5919771745985626 is not the same as expected 0.610803
DISS_f10_D failed - 0.9886580780373544 is not the same as expected 0.988843
CT_f_e_conn passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_e_conn passed
CT_e_D_An failed - 435 is not the same as expected 419
CT_e_A_Ca failed - 3523 is not the same as expected 3658
CT_f_D_tort1 failed - 0.1194600258891697 is not the same as expected 0.890216
CT_f_A_tort1 failed - 0.2667435538347475 is not the same as expected 0.265837
CT_n_D_adj_An passed
CT_n_A_adj_Ca passed
Total time to calculate graph: 9.780888557434082 second(s)
Total time to calculate descriptors: 9.515177965164185 second(s)
Peak memory usage for descriptor calculation: 53702457 bytes
{'STAT_n': 40501, 'STAT_e': 3523, 'STAT_n_D': 16223, 'STAT_n_A': 24278, 'STAT_CC_D': 43, 'STA

In [4]:
import combined_without as ig
import new_descriptors as ds
import os
import time

current_dir = os.getcwd()
data_path = f"{current_dir}/graspi_igraph/data/"
descriptors_path = f"{current_dir}/graspi_igraph/descriptors/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5
for test_file in test_files:
    if 'data_4_3_2' in test_file or 'data_5x4x3' in test_file:
        continue
    # g = ig.generateGraph(data_path + test_file + ".txt")
    import time
    graph_start = time.time()
    g,is_2D, black_vertices, white_vertices, black_green, black_interface_red, white_interface_blue, dim, interface_edge_comp_paths, shortest_path_to_red, shortest_path_to_blue, CT_n_D_adj_An, CT_n_A_adj_Ca= ig.generateGraph(data_path + test_file + ".txt")
    graph_end = time.time()
    stats = ds.descriptors(g, test_file, black_vertices, white_vertices, black_green, black_interface_red, white_interface_blue, dim, interface_edge_comp_paths, shortest_path_to_red, shortest_path_to_blue, CT_n_D_adj_An, CT_n_A_adj_Ca)
    #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    with open(descriptors_path + "descriptors." + test_file + ".log") as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
    time = stats["time"]
    mem = stats["mem"]
    graph_time = graph_end-graph_start
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {time} second(s)")
    print(f"Peak memory usage for descriptor calculation: {mem} bytes")
    print(stats)
    print("")

data_0.615_4.0_000160 Results
STAT_n passed
STAT_e passed
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_f_D passed
DISS_wf10_D failed - 0.588312680039658 is not the same as expected 0.606972
DISS_f10_D passed
CT_f_e_conn passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_e_conn passed
CT_e_D_An failed - 432 is not the same as expected 416
CT_e_A_Ca failed - 3466 is not the same as expected 3599
CT_f_D_tort1 failed - 0.1194600258891697 is not the same as expected 0.890216
CT_f_A_tort1 failed - 0.2667435538347475 is not the same as expected 0.265837
CT_n_D_adj_An passed
CT_n_A_adj_Ca passed
Total time to calculate graph: 9.817759037017822 second(s)
Total time to calculate descriptors: 9.37238335609436 second(s)
Peak memory usage for descriptor calculation: 53603776 bytes
{'STAT_n': 40501, 'STAT_e': 3466, 'STAT_n_D': 16223, 'STAT_n_A': 24278, 'STAT_CC_D': 46, 'STAT_CC_A': 1, 'STAT_CC_D_An': 8, 'STAT_CC_A_Ca': 1, 'ABS_f_D':